In [5]:
import os
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
sys.path.append(os.pardir)
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from models.word_embedding import get_embedding_matrix
from models.cnn_kr import CNNClassifier

In [3]:
w2v_model_name = '../model_save/embedding_model/Word2vec1.model'
word_vectorizer = Word2Vec.load(w2v_model_name)

pre_trained_name = '../model_save/embedding_model/trained_word2vec1.model'
pre_trained_w2v = Word2Vec.load(pre_trained_name)

## 01. Data Load

In [21]:
DATA_IN_PATH = '../assets/data/npy_data/2020-05-31/'

# Data save label
TRAIN_INPUT_DATA = 'train_input.npy'
TEST_INPUT_DATA = 'test_input.npy'
DATA_CONFIGS = 'data_configs.json'
SEQ_CONFIGS = 'seq_configs_bt.json'

# Train label save file name
TRAIN_LABEL_DATA = 'train_label.npy'
TEST_LABEL_DATA = 'test_label.npy'
TRAIN_LABEL_SMALL = 'train_label_small.npy'
TEST_LABEL_SMALL = 'test_label_small.npy'

In [22]:
train_X = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
train_Y = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
test_X = np.load(open(DATA_IN_PATH + TEST_INPUT_DATA, 'rb'))
test_Y = np.load(open(DATA_IN_PATH + TEST_LABEL_DATA, 'rb'))
train_YS = tf.one_hot(train_Y, 43)
test_YS = tf.one_hot(test_Y, 43)
data_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))
vocab_size = data_configs['vocab_size'] + 1
print("vocab_size : ", vocab_size)
word_index = data_configs['vocab']

vocab_size :  30079


### best parameter each neural network

In [23]:
cnn_kargs = {'vocab_size': vocab_size,
             'embedding_size': 300, 
             'num_filters': 128, 
             'dropout_rate': 0.5, 
             'hidden_dimension': 500, 
             'train_mode': 'non_static', 
             'output_dimension': 43, 
             'trainable':True,
             'optimizer':'adam',
             'model_name': 'cnn_non_static_adam_w2v'}

In [24]:
rnn_kargs = {'vocab_size': vocab_size,
             'embedding_size': 300, 
             'dropout_rate': 0.5, 
             'lstm_dimension': 128, 
             'dense_dimension': 64, 
             'train_mode': 'pt', 
             'output_dimension': 43, 
             'optimizer':'radam',
             'model_name': 'lstm_pt_radam_pt_w2v'}

## 02. model evaluation

### 02-1. big_class

In [25]:
from models.model_evaluation import model_build, return_two_calss_acc

### cnn_non_static w2v

In [26]:
cnn_model = model_build(train_X, word_index, word_vectorizer, **cnn_kargs)

word index size :  30079
27423 2655
cnn
big_class
Model: "cnn_non_static_adam_w2v"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  9023700   
_________________________________________________________________
conv1d_6 (Conv1D)            multiple                  115328    
_________________________________________________________________
conv1d_7 (Conv1D)            multiple                  153728    
_________________________________________________________________
conv1d_8 (Conv1D)            multiple                  192128    
_________________________________________________________________
global_max_pooling1d_2 (Glob multiple                  0         
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
___________________________________________________________

In [27]:
cnn_model.evaluate(test_X, test_YS)

145/145 [==============================] - 1s 5ms/step - loss: 0.6180 - accuracy: 0.8733 - f1_score: 0.8592


[0.6180410981178284, 0.8732699155807495, 0.8592448830604553]

### predict_proba two class

In [14]:
two_class_acc = return_two_calss_acc(cnn_model, test_X, test_Y)

In [15]:
two_class_acc

0.9463667820069204

### bidirection LSTM

In [16]:
rnn_model = model_build(train_X, word_index, word_vectorizer, **rnn_kargs)

word index size :  30079
27423 2655
rnn
big_class
Model: "lstm_pt_radam_pt_w2v"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  9023700   
_________________________________________________________________
bidirectional (Bidirectional multiple                  439296    
_________________________________________________________________
bidirectional_1 (Bidirection multiple                  394240    
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  16448     
_________________________________________________________________
dense_3 (Dense)              multiple                  2795      
Total params: 9,876,479
Trainable params: 9,876,479
Non-traina

In [22]:
rnn_model.evaluate(test_X, test_YS)

145/145 [==============================] - 2s 17ms/step - loss: 0.8131 - accuracy: 0.8443 - f1_score: 0.8163


[0.8131087422370911, 0.8442906737327576, 0.8162685632705688]

In [23]:
two_class_acc = return_two_calss_acc(rnn_model, test_X, test_Y)

In [24]:
two_class_acc

0.9154411764705882

### AC-BLSTM

### 02-2. small_class

### CNN

In [32]:
test_Y = np.load(open(DATA_IN_PATH + TEST_LABEL_SMALL, 'rb'))
test_YS = tf.one_hot(test_Y, 455)

In [33]:
cnn_kargs['output_dimension'] = 455
rnn_kargs['output_dimension'] = 455

In [34]:
cnn_model_small = model_build(train_X, word_index, word_vectorizer, **cnn_kargs)

word index size :  30079
27423 2655
cnn
small_class
Model: "cnn_non_static_adam_w2v"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  9023700   
_________________________________________________________________
conv1d_6 (Conv1D)            multiple                  115328    
_________________________________________________________________
conv1d_7 (Conv1D)            multiple                  153728    
_________________________________________________________________
conv1d_8 (Conv1D)            multiple                  192128    
_________________________________________________________________
global_max_pooling1d_2 (Glob multiple                  0         
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________

In [35]:
cnn_model_small.evaluate(test_X, test_YS)

145/145 [==============================] - 1s 5ms/step - loss: 1.3026 - accuracy: 0.7260 - f1_score: 0.6748


[1.3026211261749268, 0.7259948253631592, 0.6747869849205017]

In [36]:
two_class_acc_small = return_two_calss_acc(cnn_model_small, test_X, test_Y)

In [37]:
two_class_acc_small

0.8365051903114187

### LSTM

In [38]:
rnn_model_small = model_build(train_X, word_index, word_vectorizer, **rnn_kargs)

word index size :  30079
27423 2655
rnn
small_class
Model: "lstm_pt_radam_pt_w2v"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  9023700   
_________________________________________________________________
bidirectional_2 (Bidirection multiple                  439296    
_________________________________________________________________
bidirectional_3 (Bidirection multiple                  394240    
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  16448     
_________________________________________________________________
dense_9 (Dense)              multiple                  29575     
Total params: 9,903,259
Trainable params: 9,903,259
Non-trai

In [39]:
rnn_model_small.evaluate(test_X, test_YS)

145/145 [==============================] - 2s 17ms/step - loss: 1.3292 - accuracy: 0.6741 - f1_score: 0.5905


[1.3292149305343628, 0.6740916967391968, 0.5904927253723145]

In [40]:
two_class_acc_small = return_two_calss_acc(rnn_model_small, test_X, test_Y)

In [41]:
two_class_acc_small

0.8016868512110726

### 03. application